In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

In [2]:
def draftkings_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    odds = [x.text for x in soup.findAll('span',{'class', input_dict['odds_class']})]
    # odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::3], outcomes[2::3], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

def fanduel_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[::3], odds[1::3], odds[2::3], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

def bovada_parse(input_dict, driver, key):
    outcomes = [x.text for x in driver.find_elements_by_class_name(input_dict['outcomes_class'])]
    odds = [x.text for x in driver.find_elements_by_class_name(input_dict['odds_class'])]
    return list(zip(outcomes[::2], outcomes[1::2], odds[2::7], odds[3::7], odds[4::7], [key] * len(outcomes[::3]), [pd.datetime.now()] * len(outcomes[::3])))

In [3]:
prem_dir = {'draftkings':{'url':'https://sportsbook.draftkings.com/leagues/soccer/53591936',
                          'parser': draftkings_parse,
                          'args':{'outcomes_class':'sportsbook-outcome-cell__label',
                                  'odds_class':'sportsbook-odds american default-color'}},
            'fanduel':{'url':'https://sportsbook.fanduel.com/sports/navigation/730.1/7567.1',
                       'parser': fanduel_parse,
                       'args':{'outcomes_class':'name',
                               'odds_class':'selectionprice'}},
            'bovada':{'url':'https://www.bovada.lv/sports/soccer/england-premier-league',
                      'parser': bovada_parse,
                      'args':{'outcomes_class':'name',
                              'odds_class':'bet-price'}}
           }
           

In [5]:
options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")

DRIVER_PATH = 'driver/chromedriver'
column_names = ['home_team','away_team', 'home_odds','tie_odds', 'away_odds', 'source', 'datetime']

for key in prem_dir:    
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get(prem_dir[key]['url'])
    driver.implicitly_wait(3)

    data = prem_dir[key]['parser'](prem_dir[key]['args'],driver,key)
    print(pd.DataFrame(data, columns=column_names).head())
    
    driver.quit()

           home_team         away_team home_odds tie_odds away_odds  \
0     Leicester City          Brighton      -148     +285      +410   
1          Tottenham          West Ham      -155     +310      +410   
2  Manchester United  Sheffield United      -240     +335      +750   
3          Newcastle       Aston Villa      +130     +230      +220   
4             Wolves       Bournemouth      -182     +320      +510   

       source                   datetime  
0  draftkings 2020-06-22 17:51:08.995727  
1  draftkings 2020-06-22 17:51:08.995727  
2  draftkings 2020-06-22 17:51:08.995727  
3  draftkings 2020-06-22 17:51:08.995727  
4  draftkings 2020-06-22 17:51:08.995727  
   home_team    away_team home_odds tie_odds away_odds   source  \
0  Leicester     Brighton      -145     +270      +440  fanduel   
1  Tottenham     West Ham      -155     +290      +420  fanduel   
2  Newcastle  Aston Villa      +135     +230      +210  fanduel   
3     Wolves  Bournemouth      -180     +310   